In [ ]:
# Add feature for defendants who are detained/released. Detainees are very unlikely to FTA because appearance is compulsory.

In [34]:
import pandas as pd
import os
import featuretools as ft

In [2]:
os.getcwd()

'C:\\Users\\TJ\\OneDrive - The George Washington University\\Capstone\\FTA-Modeling\\notebooks'

In [3]:
os.chdir("..\\..\\fta") # changed directory to fta access data files

In [4]:
events = pd.read_csv('events_data_2015.csv')
summary = pd.read_csv('summary_data_2015.csv')

In [5]:
# Identify all detention statuses
detention = ['Preventive Detention', 'Defendant detained pending sentencing','Defendant Held in Other Jurisdiction']
events['detention'] = events['Result'].apply(lambda x: 1 if x in detention else 0 )
events['event_id'] = events.index

In [14]:
events.shape

(25329, 8)

In [6]:
events.head(5)

DateTime        Location                      Type  \
0   1/2/2015 1:00 PM  Courtroom C-10               Presentment   
1   1/5/2015 9:00 AM   Courtroom 301       Preliminary Hearing   
2  1/15/2015 9:30 AM   Courtroom 217  Felony Status Conference   
3  2/12/2015 9:30 AM   Courtroom 217        Felony Arraignment   
4  2/23/2015 9:30 AM   Courtroom 217  Felony Status Conference   

                                             Result Judge          case_id  \
0                              Preventive Detention   NaN  2015 CF2 000024   
1          Probable Cause Found Held for Grand Jury   NaN  2015 CF2 000024   
2                                      Hearing Held   NaN  2015 CF2 000024   
3  Defn Arraigned Plead Not Guilty Jury Demand Made   NaN  2015 CF2 000024   
4                                   Hearing Vacated   NaN  2015 CF2 000024   

   detention  event_id  
0          1         0  
1          0         1  
2          0         2  
3          0         3  
4          0         4

In [13]:
events.isna().sum()

DateTime         0
Location         0
Type             0
Result          29
Judge        15710
case_id          0
detention        0
event_id         0
dtype: int64

In [7]:
summary.head()

case_id                                         case_title  \
0  2015 CF2 000024  2015 CF2 000024          United States  Vs.  L...   
1  2015 CF2 000039  2015 CF2 000039 United States  Vs. BLUNT, CURT...   
2  2015 CF2 000041  2015 CF2 000041 United States  Vs. JOHNSON, DA...   
3  2015 CF2 000042  2015 CF2 000042 United States  Vs. PRESSLEY, E...   
4  2015 CF2 000066  2015 CF2 000066 United States  Vs. COLEMAN, BA...   

                party               party_aka           party_attny  
0  LITTLE, JR, MELVIN        LITTLE, MELVIN N   MCDANIEL, BRIAN K K  
1     BLUNT, CURTIS E    BLUNT, CURTIS EDWARD      BAER, MITCHELL S  
2    JOHNSON, DARIOUS         JOHNSON, DARIUS       NICHOLS, ARCHIE  
3     PRESSLEY, EDDIE   PRESSLEY, EDDIE DEVON       HARRIS, JULLIAN  
4    COLEMAN, BARRY D  COLEMAN, BARRY DEWAYNE  PHILLIPS, KIMBERLY J

In [11]:
summary.isna().sum()

case_id          0
case_title       0
party            0
party_aka      701
party_attny      3
dtype: int64

In [18]:
# drop data with party_aka is null since it's the distinguishing factor between similar names
summary.dropna(subset=["party_aka"], inplace=True)

In [19]:
# create unique index for each party
data = pd.merge(events, summary, on= 'case_id')

data['DateTime'] = pd.to_datetime(data['DateTime'])

data.head()

DateTime        Location                      Type  \
0 2015-01-02 13:00:00  Courtroom C-10               Presentment   
1 2015-01-05 09:00:00   Courtroom 301       Preliminary Hearing   
2 2015-01-15 09:30:00   Courtroom 217  Felony Status Conference   
3 2015-02-12 09:30:00   Courtroom 217        Felony Arraignment   
4 2015-02-23 09:30:00   Courtroom 217  Felony Status Conference   

                                             Result Judge          case_id  \
0                              Preventive Detention   NaN  2015 CF2 000024   
1          Probable Cause Found Held for Grand Jury   NaN  2015 CF2 000024   
2                                      Hearing Held   NaN  2015 CF2 000024   
3  Defn Arraigned Plead Not Guilty Jury Demand Made   NaN  2015 CF2 000024   
4                                   Hearing Vacated   NaN  2015 CF2 000024   

   detention  event_id                                         case_title  \
0          1         0  2015 CF2 000024          United States  Vs.  L...   
1          0         1  2015 CF2 000024          United States  Vs.  L...   
2          0         2  2015 CF2 000024          United States  Vs.  L...   
3          0         3  2015 CF2 000024          United States  Vs.  L...   
4          0         4  2015 CF2 000024          United States  Vs.  L...   

                party         party_aka          party_attny  
0  LITTLE, JR, MELVIN  LITTLE, MELVIN N  MCDANIEL, BRIAN K K  
1  LITTLE, JR, MELVIN  LITTLE, MELVIN N  MCDANIEL, BRIAN K K  
2  LITTLE, JR, MELVIN  LITTLE, MELVIN N  MCDANIEL, BRIAN K K  
3  LITTLE, JR, MELVIN  LITTLE, MELVIN N  MCDANIEL, BRIAN K K  
4  LITTLE, JR, MELVIN  LITTLE, MELVIN N  MCDANIEL, BRIAN K K

In [9]:
data.shape

(25329, 12)

In [20]:
data.shape

(16901, 12)

In [10]:
# check number of nas
data.isna().sum()

DateTime           0
Location           0
Type               0
Result            29
Judge          15710
case_id            0
detention          0
event_id           0
case_title         0
party              0
party_aka       8428
party_attny       22
dtype: int64

In [21]:
# check number of nas
data.isna().sum()

DateTime           0
Location           0
Type               0
Result            21
Judge          10352
case_id            0
detention          0
event_id           0
case_title         0
party              0
party_aka          0
party_attny        8
dtype: int64

In [23]:
# Create unique index for each person based on concatenation of party and party_aka (in the meantime)

data['consumer'] = data['party'] + data['party_aka']
data['consumer_id'] = data['consumer'].astype("category").cat.codes

data.tail()

DateTime       Location                               Type  \
16896 2016-09-24 13:30:00  Courtroom 114  Safe Surrender Initial Appearance   
16897 2016-10-07 09:30:00  Courtroom 319           Felony Status Conference   
16898 2016-10-20 09:30:00  Courtroom 319           Felony Status Conference   
16899 2016-12-07 09:30:00  Courtroom 319                 Felony Arraignment   
16900 2017-04-25 09:30:00  Courtroom 319                         Jury Trial   

                                                 Result               Judge  \
16896                                      Hearing Held  BESHOURI, JOSEPH E   
16897                       Defense Counsel Unavailable  BESHOURI, JOSEPH E   
16898                                      Hearing Held  BESHOURI, JOSEPH E   
16899  Defn Arraigned Plead Not Guilty Jury Demand Made  BESHOURI, JOSEPH E   
16900                                 Dismissed - Nolle  KNOWLES, KIMBERLEY   

               case_id  detention  event_id  \
16896  2015 CF2 018194          0     25324   
16897  2015 CF2 018194          0     25325   
16898  2015 CF2 018194          0     25326   
16899  2015 CF2 018194          0     25327   
16900  2015 CF2 018194          0     25328   

                                              case_title               party  \
16896  2015 CF2 018194          United States  Vs.  S...  STEWART, MICHAEL K   
16897  2015 CF2 018194          United States  Vs.  S...  STEWART, MICHAEL K   
16898  2015 CF2 018194          United States  Vs.  S...  STEWART, MICHAEL K   
16899  2015 CF2 018194          United States  Vs.  S...  STEWART, MICHAEL K   
16900  2015 CF2 018194          United States  Vs.  S...  STEWART, MICHAEL K   

                     party_aka         party_attny  \
16896  STEWART, MICHAEL KESHON  ROBERTSON, KEVIN C   
16897  STEWART, MICHAEL KESHON  ROBERTSON, KEVIN C   
16898  STEWART, MICHAEL KESHON  ROBERTSON, KEVIN C   
16899  STEWART, MICHAEL KESHON  ROBERTSON, KEVIN C   
16900  STEWART, MICHAEL KESHON  ROBERTSON, KEVIN C   

                                        consumer  consumer_id  
16896  STEWART, MICHAEL KSTEWART, MICHAEL KESHON         1045  
16897  STEWART, MICHAEL KSTEWART, MICHAEL KESHON         1045  
16898  STEWART, MICHAEL KSTEWART, MICHAEL KESHON         1045  
16899  STEWART, MICHAEL KSTEWART, MICHAEL KESHON         1045  
16900  STEWART, MICHAEL KSTEWART, MICHAEL KESHON         1045

In [25]:
# Select subset of individuals with multiple cases maybe?


data.columns

Index(['DateTime', 'Location', 'Type', 'Result', 'Judge', 'case_id',
       'detention', 'event_id', 'case_title', 'party', 'party_aka',
       'party_attny', 'consumer', 'consumer_id'],
      dtype='object')

In [ ]:
# Method 1

In [29]:
# Create dataframe for consumer
columns = ['consumer', 'consumer_id']
consumer = data[columns].copy()

In [30]:
consumer.drop_duplicates(inplace = True)

In [33]:
consumer.shape

(1256, 2)

In [55]:
# Create entity set
es = ft.EntitySet(id = 'consumer')

In [56]:
# Create an entity from the consumer dataframe

es = es.entity_from_dataframe(entity_id = 'consumer', 
                              dataframe = consumer, 
                              index = 'consumer_id', 
                             variable_types = {
                               'consumer_id': ft.variable_types.Categorical,  
                                 
                             })

In [57]:
# Create an entity from the data dataframe
# This does not yet have a unique index

es = es.entity_from_dataframe(entity_id = 'court_info', 
                              dataframe = data,
                              variable_types = {
                                  'event_id': ft.variable_types.Categorical,
                                  'consumer_id': ft.variable_types.Categorical,
                                  'detention': ft.variable_types.Boolean,
                              },
                              index = 'event_id',
                              time_index = 'DateTime')

In [58]:
es.__dict__

{'id': 'consumer', 'entity_dict': {'consumer': Entity: consumer
    Variables:
      consumer_id (dtype: index)
      consumer (dtype: categorical)
    Shape:
      (Rows: 1256, Columns: 2), 'court_info': Entity: court_info
    Variables:
      event_id (dtype: index)
      DateTime (dtype: datetime_time_index)
      Location (dtype: categorical)
      Type (dtype: categorical)
      Result (dtype: categorical)
      Judge (dtype: categorical)
      case_id (dtype: categorical)
      case_title (dtype: text)
      party (dtype: categorical)
      party_aka (dtype: categorical)
      party_attny (dtype: categorical)
      consumer (dtype: categorical)
      consumer_id (dtype: categorical)
      detention (dtype: boolean)
    Shape:
      (Rows: 16901, Columns: 14)}, 'relationships': [], 'time_type': featuretools.variable_types.variable.DatetimeTimeIndex, '_data_description': None}

In [59]:
es['consumer']

Entity: consumer
  Variables:
    consumer_id (dtype: index)
    consumer (dtype: categorical)
  Shape:
    (Rows: 1256, Columns: 2)

In [60]:
es['court_info']

Entity: court_info
  Variables:
    event_id (dtype: index)
    DateTime (dtype: datetime_time_index)
    Location (dtype: categorical)
    Type (dtype: categorical)
    Result (dtype: categorical)
    Judge (dtype: categorical)
    case_id (dtype: categorical)
    case_title (dtype: text)
    party (dtype: categorical)
    party_aka (dtype: categorical)
    party_attny (dtype: categorical)
    consumer (dtype: categorical)
    consumer_id (dtype: categorical)
    detention (dtype: boolean)
  Shape:
    (Rows: 16901, Columns: 14)

In [61]:
# Create relationship between consumer and court information
r_consumer_information = ft.Relationship(es['consumer']['consumer_id'],
                                         es['court_info']['consumer_id'])

# Add relationship to entity set
es = es.add_relationship(r_consumer_information)

In [62]:
es

Entityset: consumer
  Entities:
    consumer [Rows: 1256, Columns: 2]
    court_info [Rows: 16901, Columns: 14]
  Relationships:
    court_info.consumer_id -> consumer.consumer_id

In [63]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='consumer', 
                                 max_depth = 2)


In [64]:
features.head()

consumer  COUNT(court_info)  \
consumer_id                                                              
0            ABBOTT, DEANDREABBOTT, DEANDRE ANTONIO                 10   
1                     ABEY, JASON ANTWANABEY, JASON                  2   
2                        ABNEY, LAVOZABNEY, LAVOZ J                  7   
3                     ACKER, JESSIE RACKER, JESSE R                 15   
4               ADAMS, MICHAELADAMS, MICHAEL CALVIN                  5   

             PERCENT_TRUE(court_info.detention)  \
consumer_id                                       
0                                      0.100000   
1                                      0.000000   
2                                      0.142857   
3                                      0.000000   
4                                      0.000000   

             NUM_UNIQUE(court_info.Location)  NUM_UNIQUE(court_info.Type)  \
consumer_id                                                                 
0                                          3                            8   
1                                          2                            2   
2                                          3                            4   
3                                          5                            6   
4                                          4                            5   

             NUM_UNIQUE(court_info.Result)  NUM_UNIQUE(court_info.Judge)  \
consumer_id                                                                
0                                        9                             0   
1                                        2                             2   
2                                        6                             1   
3                                        8                             0   
4                                        5                             0   

             NUM_UNIQUE(court_info.case_id)  NUM_UNIQUE(court_info.party)  \
consumer_id                                                                 
0                                         1                             1   
1                                         1                             1   
2                                         1                             1   
3                                         1                             1   
4                                         1                             1   

             NUM_UNIQUE(court_info.party_aka)  ...  \
consumer_id                                    ...   
0                                           1  ...   
1                                           1  ...   
2                                           1  ...   
3                                           1  ...   
4                                           1  ...   

             MEAN(court_info.NUM_WORDS(case_title))  \
consumer_id                                           
0                                               9.0   
1                                              10.0   
2                                              25.0   
3                                              10.0   
4                                               9.0   

             MEAN(court_info.NUM_CHARACTERS(case_title))  \
consumer_id                                                
0                                                   50.0   
1                                                   53.0   
2                                                   66.0   
3                                                   50.0   
4                                                   49.0   

            NUM_UNIQUE(court_info.DAY(DateTime))  \
consumer_id                                        
0                                              8   
1                                              2   
2                                              7   
3                                             12   
4                                              4   

  

In [65]:
feature_names

[<Feature: consumer>,
 <Feature: COUNT(court_info)>,
 <Feature: PERCENT_TRUE(court_info.detention)>,
 <Feature: NUM_UNIQUE(court_info.Location)>,
 <Feature: NUM_UNIQUE(court_info.Type)>,
 <Feature: NUM_UNIQUE(court_info.Result)>,
 <Feature: NUM_UNIQUE(court_info.Judge)>,
 <Feature: NUM_UNIQUE(court_info.case_id)>,
 <Feature: NUM_UNIQUE(court_info.party)>,
 <Feature: NUM_UNIQUE(court_info.party_aka)>,
 <Feature: NUM_UNIQUE(court_info.party_attny)>,
 <Feature: NUM_UNIQUE(court_info.consumer)>,
 <Feature: MODE(court_info.Location)>,
 <Feature: MODE(court_info.Type)>,
 <Feature: MODE(court_info.Result)>,
 <Feature: MODE(court_info.Judge)>,
 <Feature: MODE(court_info.case_id)>,
 <Feature: MODE(court_info.party)>,
 <Feature: MODE(court_info.party_aka)>,
 <Feature: MODE(court_info.party_attny)>,
 <Feature: MODE(court_info.consumer)>,
 <Feature: SUM(court_info.NUM_WORDS(case_title))>,
 <Feature: SUM(court_info.NUM_CHARACTERS(case_title))>,
 <Feature: STD(court_info.NUM_WORDS(case_title))>,
 <F